In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import graph_tool.all as gt
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from functions import *


# Graph-Tool compatibility
plt.switch_backend('cairo')
# Style
sns.set_theme(context='talk', style='white', palette='Set2')
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

# Computation

In [3]:
# Load metadata
meta = get_meta()

# Subject preview
filtered = []
for i, row in meta.iterrows():
    try:
        load_graph_by_id(row['SubID'])
        assert not np.isnan(row['nps_MoodDysCurValue'])  # Has NPS
        assert row['Sex'] == 'Female'
        assert row['Ethnicity'] != 'White'
    except:
        continue
    filtered.append(f'{row["SubID"]} {row["Ethnicity"]} {row["Sex"]}, {row["Age"]}, BRAAK {row["BRAAK_AD"]}')
for i in (2, 4, 5, 6):
    print(filtered[i])

# Parameters
column_diff = 'att_D_AD_0_1'
column_data = 'att_D_no_prior_0'
coex_diff_compare_phenotype = 'nps_PsychoAgiCurValue'
diff_data_compare_phenotype = 'nps_WtLossCurValue'
individual_subject_ids = ['M19050', 'M59593', 'M83214', 'M36634']

M19050 Hispanic Female, 74.0, BRAAK 5.0
M59593 Hispanic Female, 76.0, BRAAK 5.0
M83214 Hispanic Female, 83.0, BRAAK 6.0
M36634 Hispanic Female, 87.0, BRAAK 6.0


In [4]:
# Compute individual graphs
diff_g_individuals = [compute_graph(load_graph_by_id(individual_subject_ids[i], column=column_diff)) for i in range(len(individual_subject_ids))]
data_g_individuals = [compute_graph(load_graph_by_id(individual_subject_ids[i], column=column_data)) for i in range(len(individual_subject_ids))]

# Plots

### Individual Trio Comparison

In [5]:
# Choose three graphs
graphs = data_g_individuals[:3]
graphs_subject_ids = individual_subject_ids[:3]

# Create figure
fig, axs = get_mosaic([list(range(len(graphs)+1))], scale=9)

# Compute edge summaries
df, concatenated_graph = compute_edge_summary(graphs=graphs, subject_ids=graphs_subject_ids)

# Show individual graph comparisons
plot_graph_comparison(graphs, concatenated_graph=concatenated_graph, axs=axs, subject_ids=graphs_subject_ids)

# Show edge summary
plot_edge_summary(graphs, df=df, ax=axs[len(graphs)], subject_ids=graphs_subject_ids)

# Save figure
plt.tight_layout()
fig.savefig(f'../plots/IndividualTrioComparison.pdf', format='pdf', transparent=True, backend='cairo')

/tmp/ipykernel_862/1518908005.py:18: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### Aggregate Trio Comparison

In [8]:
from functions.computation import compute_aggregate_edge_summary

# Parameters
contrast = 'c01x'
column = column_diff

# Create figure
fig, axs = get_mosaic([list(range(len(get_contrast(contrast))+1))], scale=9)

# Compute aggregate edge summaries
contrast_group = compute_aggregate_edge_summary(get_contrast(contrast), column=column_diff)

# Plot graph comparison
plot_graph_comparison(
    graphs=[v for k, v in contrast_group[0].items()],
    subject_ids=[k for k, v in contrast_group[1].items()],
    axs=[axs[i] for i in range(len(get_contrast(contrast)))])

# Plot edge summary for subgroups
plot_aggregate_edge_summary(ax=axs[len(get_contrast(contrast))], contrast=contrast_group)

# Save figure
fig.savefig(f'../plots/AggregateTrioComparison.pdf', format='pdf', transparent=True, backend='cairo')

  6%|████▊                                                                               | 4/70 [00:00<00:03, 20.78it/s]


In [59]:
# Record edge instances
df = pd.DataFrame(columns=['Edge', 'Subgroup', 'Count'])
for subgroup in contrast_group[0]:
    g = contrast_group[0][subgroup]
    for e in g.edges():
        coefs = g.ep.coefs[e]
        df.loc[df.shape[0]] = [get_edge_string(g, e), subgroup, sum([c!=0 for c in coefs])]

# Get edge counts
count_table = df.pivot(index='Edge', columns='Subgroup', values='Count')
count_table = count_table.fillna(0)
# TODO: REVISE DIFFERENCE METRIC
count_table['Difference'] = count_table['AD'] - count_table['Control']
count_table['Range'] = count_table.max(axis=1) - count_table.min(axis=1)

# Get list of linkages by significance
for i in range(5, -5, -1):
    condition = (count_table['Difference'] >= 2)
    significant_edges = list(count_table.loc[condition].index)
    synthetic_genes = np.concatenate([detect_synthetic_vertices_graph(contrast_group[0][subgroup]) for subgroup in contrast_group[0]])
    significant_genes = np.concatenate([e.split('-') for e in significant_edges])
    significant_genes = np.unique([g for g in significant_genes if g not in synthetic_genes])

    # Print significant genes
    print(f'--- {i} ---')
    for g in significant_genes:
        print(g)
    print()

---5---
AS1
BLNK
DLX1
DLX6
EBF1
FOXC1
FOXL2
GATA3
HEYL
HLF
KCNIP1
KLF6
LEF1
LHX6
LINC01122
MAF
MAFB
MCTP2
MEF2C
MXI1
NFAT5
NR2E1
NR2F2
PGR
POU3F4
PRDM1
PRDM16
PROP1
RND3
RUNX2
RXRA
SOX10
SOX17
SOX5
SOX6
SOX8
STAT4
SYNPR
TAL1
TBX2
TBX3
TCF12
TCF4
TCF7L1
THSD4
TLE3
ZMAT4

---4---
AS1
BLNK
DLX1
DLX6
EBF1
FOXC1
FOXL2
GATA3
HEYL
HLF
KCNIP1
KLF6
LEF1
LHX6
LINC01122
MAF
MAFB
MCTP2
MEF2C
MXI1
NFAT5
NR2E1
NR2F2
PGR
POU3F4
PRDM1
PRDM16
PROP1
RND3
RUNX2
RXRA
SOX10
SOX17
SOX5
SOX6
SOX8
STAT4
SYNPR
TAL1
TBX2
TBX3
TCF12
TCF4
TCF7L1
THSD4
TLE3
ZMAT4

---3---
AS1
BLNK
DLX1
DLX6
EBF1
FOXC1
FOXL2
GATA3
HEYL
HLF
KCNIP1
KLF6
LEF1
LHX6
LINC01122
MAF
MAFB
MCTP2
MEF2C
MXI1
NFAT5
NR2E1
NR2F2
PGR
POU3F4
PRDM1
PRDM16
PROP1
RND3
RUNX2
RXRA
SOX10
SOX17
SOX5
SOX6
SOX8
STAT4
SYNPR
TAL1
TBX2
TBX3
TCF12
TCF4
TCF7L1
THSD4
TLE3
ZMAT4

---2---
AS1
BLNK
DLX1
DLX6
EBF1
FOXC1
FOXL2
GATA3
HEYL
HLF
KCNIP1
KLF6
LEF1
LHX6
LINC01122
MAF
MAFB
MCTP2
MEF2C
MXI1
NFAT5
NR2E1
NR2F2
PGR
POU3F4
PRDM1
PRDM16
PROP1
RND3
RUNX2
RXRA
SOX10


### Differentially Expressed Edges

In [ ]:
# # TODO: Fix nodes cutting off
# # Plot total and subplots for aggregate differences
# for prefix, individuals in zip(('diff', 'data'), (diff_g_individuals, data_g_individuals)):
#     plt.clf()
#     concat = concatenate_graphs(*individuals)
#     concat = get_intersection(concat)
#     concat = cull_isolated_leaves(concat)
#     concat = remove_text_by_centrality(concat)
#     concat = color_by_significance(concat)
#     visualize_graph(concat)
#     plt.gca().axis('off')
#     plt.tight_layout()
#     plt.savefig(f'../plots/{prefix}_concat.pdf', format='pdf', transparent=True, backend='cairo')

#     # Show all subsets of graph by cell type
#     for v_name in detect_synthetic_vertices_graph(concat):
#         plt.clf()
#         subset = subset_by_hub(concat, [v_name])
#         visualize_graph(subset)
#         plt.gca().axis('off')
#         plt.tight_layout()
#         plt.savefig(f'../plots/{prefix}_concat_{v_name}.pdf', format='pdf', transparent=True, backend='cairo')

### Enrichment

In [ ]:
# from functions.plotting import plot_enrichment

# # Generate fake enrichment data
# import itertools
# # Get columns
# cell_type = ['OPC', 'Micro', 'Oligo', 'Endo']
# disease = ['Alzheimer\'s Disease', 'Schizophrenia', 'Bipolar Disorder', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
# combined = [val for val in itertools.product(cell_type, disease)]
# cell_type = [val[0] for val in combined]
# disease = [val[1] for val in combined]
# # Get significance
# np.random.seed(42)
# significance = np.exp(-8 * np.random.rand(len(combined)))
# # Combine
# df = pd.DataFrame({'cell_type': cell_type, 'disease': disease, 'significance': significance})
# df = df.loc[df['significance'] < 5e-2]

# # Rename
# df = df.rename(columns={'cell_type': 'Cell Type', 'disease': 'Disease'})
# # Add significance scale
# df['-log10(p)'] = -np.log10(df['significance'])
# plt.clf()
# plot_enrichment(df)
# plt.tight_layout()
# plt.savefig(f'../plots/enrichment.pdf', format='pdf', transparent=True, backend='cairo')

### Individual Edge Comparison

In [ ]:
# # NOTE: Manually annotate individual points/genes of interest
# for prefix, individuals in zip(('diff', 'data'), (diff_g_individuals, data_g_individuals)):
#     # Get graph
#     sample_ids = individual_sample_ids[:len(individuals)]
#     g = concatenate_graphs(*individuals)
#     g = get_intersection(g)
#     g = cull_isolated_leaves(g)

#     plt.clf()
#     df = plot_individual_edge_comparison(g, sample_ids)
#     plt.tight_layout()
#     plt.savefig(f'../plots/{prefix}_edge_comparison.pdf', format='pdf', transparent=True, backend='cairo')

### Legend

In [ ]:
# Plot legend
plt.clf()
plot_legend()
plt.gca().axis('off')
plt.tight_layout()
plt.savefig(f'../plots/legend.pdf', format='pdf', transparent=True, backend='cairo')

/tmp/ipykernel_786/3582710711.py:5: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
